# RAG (Retrieval Augmented Generation) System Using Llama2 With Hugging Face

In [1]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.5 MB/s eta 0:00:00


- pypdf is used for uploading all the pdf docs.

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.1 MB/s eta 0:00:00


- bitsandbytes is used in the quantization process means most of the LLM model will be in the 16 bits but since we are working on the Google Colab we will be quantized to 4 bits.
- transformers will be used to create the pipeline which will be able to use Llama2 model with respect to any kind of input that we will specifically gives.
- accelerate will speed in the process of uploading or loading the entire pipeline itself.

In [3]:
# Embedding:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.9 MB/s eta 0:00:00


In [4]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [5]:
# Importing all the libraries
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext # VectorStoreIndex is for indexing purpose, ServiceContext helps to combine llama2 model along with prompt template
from llama_index.llms import HuggingFaceLLM # helps to interact with hugging face hub where you will be able to call the entire llama2 model
from llama_index.prompts.prompts import SimpleInputPrompt

In [6]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='1a702e93-bd70-4209-930e-1eef61213c75', embedding=None, metadata={'page_label': '1', 'file_name': 'Attention.pdf', 'file_path': '/content/data/Attention.pdf', 'file_type': 'application/pdf', 'file_size': 254261, 'creation_date': '2024-02-09', 'last_modified_date': '2024-02-09', 'last_accessed_date': '2024-02-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/274699544\nAttention and Perception\nChapt er · Mar ch 2015\nDOI: 10.1002/9781118900772. etrds0018\nCITATIONS\n4READS\n14,083\n1 author:\nRonald A R ensink\nUniv ersity of British Columbia\n141 PUBLICA TIONS \xa0\xa0\xa012,021  CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll

In [9]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
# Default format supportable by Llama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [10]:
# Hugging Face Login
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [18]:
# Calling the Llama2 model from Hugging Face
import torch

llm=HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False}, # temperature value ranges between 0 to 1. It indicate the creativity of your model.
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True} # Quantizing from 16 bit to 8 bit
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

# Combing langchain with llama2
embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [20]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [21]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a38924b1240>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a38924b1240>, id_func=<function default_id_func at 0x7a3944f670a0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7a3898bc8790>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a38924b1240>)

In [22]:
# Converting the entire data into indexes
index=VectorStoreIndex.from_documents(documents, service_context=service_context)

In [23]:
index

In [24]:
# Converting the above index into query engine
query_engine=index.as_query_engine()

In [27]:
response=query_engine.query("How YOLO Algorithm works?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [29]:
print(response)

YOLO algorithm works by first separating the input image into a S X S size grids. Each grid cell tries object detection on itself, and the predicted confidence scores reflect how confident it is to provide each label and bounding box to each object. The unified detection outputs each confidence score, which includes the (x, y) coordinates representing the center of the box with respect to the grid cell's boundaries. The estimated object's height and width are calculated with respect to the entire image.


In [30]:
response=query_engine.query("What are the different kinds of attention?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [31]:
print(response)

There are several different kinds of attention, including:

1. Sustained attention: the ability to focus on a task for a prolonged period of time.
2. Selective attention: the ability to focus on a specific task or stimulus while ignoring others.
3. Alternating attention: the ability to switch focus between two or more tasks or stimuli.
4. Dividing attention: the ability to perform multiple tasks simultaneously.
5. Executive attention: the ability to control and manage attention, including the ability to switch between tasks and to focus on a specific task.
6. Attentional control: the ability to regulate and manage attention, including the ability to focus on a specific task and to switch between tasks.
7. Attentional flexibility: the ability to switch between different tasks and to adapt to changing conditions.
8. Attentional capacity: the ability to process and store information in working memory, which is the ability to hold and manipulate information in one's mind for a short period